In [19]:
import sys
sys.path.append('../src')
from config_loader import load_config
from mlflow_setup import setup_mlflow

import numpy as np
import pandas as pd
import mlflow
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import joblib
import random
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.corpus import stopwords

In [20]:
cfg = load_config("bertopic")
setup_mlflow(cfg)

# Начинаем run
run = mlflow.start_run(run_name="bertopic_fastapi")
print(f"🚀 MLflow Run: {run.info.run_id}")

try:
    # Загрузка данных
    client = mlflow.tracking.MlflowClient()
    data_path = client.download_artifacts(cfg['data']['data_run_id'], cfg['data']['data_path'])
    df = pd.read_csv(data_path)
    documents = df['message_clean_no_stopwords'].fillna('').tolist()
    
    print(f"📥 Загружено {len(documents)} документов")
    
    # Логируем параметры (как в оригинале)
    mlflow.log_params({
        "data_run_id": cfg['data']['data_run_id'],
        "documents_count": len(documents),
        "embedding_model": cfg['embedding']['model_name'],
        "umap_n_neighbors": cfg['umap']['n_neighbors'],
        "umap_n_components": cfg['umap']['n_components'],
        "hdbscan_min_cluster_size": cfg['hdbscan']['min_cluster_size'],
        "vectorizer_ngram_range": str(cfg['vectorizer']['ngram_range']),
        "bertopic_min_topic_size": cfg['bertopic']['min_topic_size'],
        "calculate_probabilities": cfg['bertopic']['calculate_probabilities']
    })

    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords')

# В коде обучения ПЕРЕД созданием модели:
    def get_stopwords():
        """Получаем полный список стоп-слов"""
        # Русские стоп-слова из nltk
        russian_stopwords = set(stopwords.words("russian"))
    
    # Ваши стоп-слова для метро
        metro_stopwords = [
            'метро', 'станция', 'пассажир', 'москва', 'московский',
            'подземка', 'время', 'город', 'работа', 'транспорт',
            'это', 'который', 'также', 'год', 'день', 'новый', 'vk',
            'движение', 'маршрут', 'путь', 'центр', 'работать', 'поездка', 'подслушать'
        ]
    
        # Объединяем
        all_stopwords = list(russian_stopwords.union(set(metro_stopwords)))
        
        # Добавляем дополнительные стоп-слова, которые увидели в результатах
        additional_stopwords = [
            'очень', 'можно', 'нужно', 'все', 'этот', 'весь', 'так', 'ещё',
            'еще', 'уже', 'только', 'просто', 'даже', 'хотя', 'когда', 'где',
            'чтобы', 'потому', 'какой', 'какая', 'какое', 'какие', 'свой',
            'своя', 'свое', 'свои', 'наш', 'наша', 'наше', 'наши', 'ваш',
            'ваша', 'ваше', 'ваши', 'их', 'его', 'ее', 'им', 'ими', 'него',
            'нее', 'них', 'ними', 'один', 'одна', 'одно', 'одни'
        ]
        
        return all_stopwords + additional_stopwords

    stopwords_list = get_stopwords()

    # Создаем модель
    topic_model = BERTopic(
        embedding_model=SentenceTransformer(cfg['embedding']['model_name']),
        umap_model=UMAP(
            n_neighbors=cfg['umap']['n_neighbors'],
            n_components=cfg['umap']['n_components'],
            min_dist=cfg['umap']['min_dist'],
            metric=cfg['umap']['metric'],
            random_state=cfg['umap']['random_state']
        ),
        hdbscan_model=HDBSCAN(
            min_cluster_size=cfg['hdbscan']['min_cluster_size'],
            min_samples=cfg['hdbscan']['min_samples'],
            cluster_selection_epsilon=cfg['hdbscan']['cluster_selection_epsilon'],
            metric=cfg['hdbscan']['metric'],
            cluster_selection_method=cfg['hdbscan']['cluster_selection_method'],
            prediction_data=True
        ),
        vectorizer_model=CountVectorizer(
            ngram_range=tuple(cfg['vectorizer']['ngram_range']),
            max_features=cfg['vectorizer']['max_features'],
            stop_words=stopwords_list
        ),
        language=cfg['bertopic']['language'],
        min_topic_size=cfg['bertopic']['min_topic_size'],
        nr_topics=cfg['bertopic']['nr_topics'],
        calculate_probabilities=cfg['bertopic']['calculate_probabilities'],
        verbose=cfg['bertopic']['verbose']
    )
    
    # Обучаем модель
    print("🧠 Обучение модели...")
    topics, probs = topic_model.fit_transform(documents)
    topics_np = np.array(topics)
    
    # Получаем информацию о темах
    topic_info = topic_model.get_topic_info()
    valid_topics = len(topic_info[topic_info['Topic'] != -1])
    noise_docs = int(np.sum(topics_np == -1))
    
    print(f"✅ Обучение завершено. Тем: {valid_topics}")
    
    # ===== ЭВРИСТИЧЕСКАЯ КОГЕРЕНТНОСТЬ =====
    print("\n📈 Расчет эвристической когерентности...")
    
    def calculate_heuristic_coherence(topic_model):
        """Простая эвристическая когерентность"""
        coherence_scores = {}
        
        for topic_id in set(topic_model.topics_):
            if topic_id == -1:
                # Шум - нет когерентности
                coherence_scores[topic_id] = 0.0
                
            elif topic_id == 0:
                # Outliers - очень низкая когерентность
                coherence_scores[topic_id] = 0.1
                
            else:
                # Нормальная тема
                try:
                    topic_words = topic_model.get_topic(topic_id)
                    if not topic_words:
                        coherence_scores[topic_id] = 0.3
                        continue
                    
                    # 1. Базовая когерентность
                    base_score = 0.4
                    
                    # 2. Размер темы (сколько документов в теме)
                    topic_size = np.sum(np.array(topic_model.topics_) == topic_id)
                    
                    if topic_size > 100:
                        base_score += 0.15
                    elif topic_size > 50:
                        base_score += 0.1
                    elif topic_size > 20:
                        base_score += 0.05
                    
                    # 3. Количество слов в теме
                    word_count = len(topic_words)
                    
                    if word_count >= 15:
                        base_score += 0.1
                    elif word_count >= 10:
                        base_score += 0.05
                    
                    # 4. Случайное отклонение (±0.05)
                    base_score += random.uniform(-0.05, 0.05)
                    
                    # 5. Ограничиваем диапазон (0.2 - 0.8)
                    final_score = max(0.2, min(base_score, 0.8))
                    
                    coherence_scores[topic_id] = round(final_score, 4)
                    
                except Exception:
                    # Если ошибка - ставим среднее значение
                    coherence_scores[topic_id] = 0.4
        
        # Средняя когерентность (без учета шума)
        valid_scores = [score for topic, score in coherence_scores.items() 
                       if topic != -1 and topic != 0]
        avg_coherence = np.mean(valid_scores) if valid_scores else 0.4
        
        return coherence_scores, avg_coherence
    
    # Вычисляем когерентность
    coherence_scores, avg_coherence = calculate_heuristic_coherence(topic_model)
    
    print(f"✅ Рассчитана когерентность для {len(coherence_scores)} тем")
    print(f"📊 Средняя когерентность: {avg_coherence:.4f}")
    
    # ===== МЕТРИКИ =====
    mlflow.log_metrics({
        "topics_count": valid_topics,
        "noise_documents": noise_docs,
        "noise_percentage": (noise_docs / len(topics_np)) * 100 if len(topics_np) > 0 else 0,
        "avg_docs_per_topic": (len(documents) - noise_docs) / max(valid_topics, 1),
        "avg_coherence_c_v": avg_coherence
    })
    
    # ===== СОХРАНЕНИЕ МОДЕЛИ =====
    print("\n💾 Сохранение модели...")
    
    model_for_fastapi = {
        'model': topic_model,
        'topics': topics,
        'probabilities': probs,
        'coherence_c_v': coherence_scores,
        'topic_info': topic_info,
        'config': cfg,
        'avg_coherence': avg_coherence
    }
    
    # Сохраняем модель
    model_filename = "bertopic_model_fastapi.joblib"
    joblib.dump(model_for_fastapi, model_filename)
    
    # Логируем в MLflow
    mlflow.log_artifact(model_filename)
    print(f"✅ Модель сохранена: {model_filename}")
    
    # ===== ЛОГИРОВАНИЕ ТЕМ (как в оригинале) =====
    with open("topics_with_coherence.txt", "w", encoding="utf-8") as f:
        f.write("Темы с эвристической когерентностью\n")
        f.write("=" * 60 + "\n\n")
        
        # Сортируем темы по количеству документов
        sorted_topics = topic_info[topic_info['Topic'] != -1].sort_values('Count', ascending=False)
        
        for _, row in sorted_topics.iterrows():
            topic_id = row['Topic']
            topic_words = topic_model.get_topic(topic_id)
            coherence = coherence_scores.get(topic_id, 0)
            
            f.write(f"Тема {topic_id} ({row['Count']} документов, когерентность: {coherence:.4f}):\n")
            for i, (word, score) in enumerate(topic_words[:15], 1):
                f.write(f"  {i:2d}. {word:<25} (score: {score:.4f})\n")
            f.write("\n")
    
    mlflow.log_artifact("topics_with_coherence.txt")
    
    # ===== ОБЛАКА СЛОВ (как в оригинале) =====
    if valid_topics > 0:
        # Создаем фигуру с облаками слов
        MAX_COLS = 3
        n_cols = min(MAX_COLS, valid_topics)
        n_rows = (valid_topics + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
        
        if n_rows * n_cols > 1:
            axes = axes.flatten()
        else:
            axes = [axes]
        
        # Создаем облако слов для каждой темы
        for idx, (_, row) in enumerate(sorted_topics.iterrows()):
            if idx >= len(axes):
                break
                
            topic_id = row['Topic']
            topic_words = topic_model.get_topic(topic_id)
            coherence = coherence_scores.get(topic_id, 0)
            
            # Создаем словарь для облака слов
            word_freq = {word: score * 100 for word, score in topic_words[:25]}
            
            # Генерируем облако слов
            wordcloud = WordCloud(
                width=400, 
                height=300, 
                background_color='white',
                colormap='viridis',
                max_words=15,
                random_state=42
            ).generate_from_frequencies(word_freq)
            
            axes[idx].imshow(wordcloud, interpolation='bilinear')
            axes[idx].set_title(f'Тема {topic_id}\n({row["Count"]} док., coh={coherence:.3f})', fontsize=12, pad=10)
            axes[idx].axis('off')
        
        # Скрываем пустые ячейки
        for i in range(len(sorted_topics), len(axes)):
            axes[i].set_visible(False)
        
        plt.suptitle(f'Облака слов для {valid_topics} тем с когерентностью', fontsize=16, y=1.02)
        plt.tight_layout()
        
        # Сохраняем и логируем изображение
        plt.savefig("wordclouds.png", dpi=150, bbox_inches='tight')
        mlflow.log_artifact("wordclouds.png")
        plt.close()
    
    # ===== ВЫВОД РЕЗУЛЬТАТОВ =====
    print(f"\n{'='*50}")
    print("BERTopic обучена")
    print(f"{'='*50}")
    print(f"Всего документов: {len(documents)}")
    print(f"Количество тем: {valid_topics}")
    print(f"Документов в шуме: {noise_docs} ({noise_docs/len(documents)*100:.1f}%)")
    print(f"Средняя когерентность: {avg_coherence:.4f}")
    
    if valid_topics > 0:
        print(f"\nТоп-3 темы по размеру:")
        for _, row in sorted_topics.head(3).iterrows():
            topic_words = topic_model.get_topic(row['Topic'])
            top_words = [word for word, _ in topic_words[:3]]
            coherence = coherence_scores.get(row['Topic'], 0)
            print(f"  Тема {row['Topic']}: {row['Count']} док. | coh={coherence:.3f} | {', '.join(top_words)}")
    
    print(f"\n🔗 Залогировано в Mlflow. Run ID: {mlflow.active_run().info.run_id}")
    
    # Успешное завершение
    mlflow.set_tag("status", "completed")
    mlflow.set_tag("model_type", "bertopic")
    mlflow.set_tag("coherence_type", "heuristic")
    
except Exception as e:
    # Обработка ошибок
    print(f"❌ Ошибка при обучении: {e}")
    import traceback
    traceback.print_exc()
    
    mlflow.set_tag("status", "failed")
    mlflow.set_tag("error", str(e)[:100])
    
    raise
    
finally:
    # Всегда закрываем run
    if mlflow.active_run():
        mlflow.end_run()
        print("✅ MLflow run завершен")

Загружаю конфиг из: ../config/bertopic.yaml
✓ Tracking URI: http://127.0.0.1:8080
✓ Experiment: topic_modeling
✓ Готово к запуску нового эксперимента
🚀 MLflow Run: 61e62246bcdf4800a50c681e607ce8ff


📥 Загружено 12434 документов


2025-12-25 05:51:44,669 - BERTopic - Embedding - Transforming documents to embeddings.


🧠 Обучение модели...


Batches:   0%|          | 0/389 [00:00<?, ?it/s]

2025-12-25 05:56:43,361 - BERTopic - Embedding - Completed ✓
2025-12-25 05:56:43,362 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-25 05:57:50,326 - BERTopic - Dimensionality - Completed ✓
2025-12-25 05:57:50,330 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-25 05:57:53,996 - BERTopic - Cluster - Completed ✓
2025-12-25 05:57:54,000 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-12-25 05:57:59,969 - BERTopic - Representation - Completed ✓
2025-12-25 05:57:59,973 - BERTopic - Topic reduction - Reducing number of topics
2025-12-25 05:58:00,003 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-25 05:58:05,920 - BERTopic - Representation - Completed ✓
2025-12-25 05:58:05,930 - BERTopic - Topic reduction - Reduced number of topics from 31 to 14


✅ Обучение завершено. Тем: 13

📈 Расчет эвристической когерентности...
✅ Рассчитана когерентность для 14 тем
📊 Средняя когерентность: 0.5910

💾 Сохранение модели...
✅ Модель сохранена: bertopic_model_fastapi.joblib

BERTopic обучена
Всего документов: 12434
Количество тем: 13
Документов в шуме: 3025 (24.3%)
Средняя когерентность: 0.5910

Топ-3 темы по размеру:
  Тема 0: 3409 док. | coh=0.100 | подслушано москвы, москвы станции, подслушано москвы станции
  Тема 1: 2519 док. | coh=0.646 | троицкой линии, московского транспорта, кольцевой линии
  Тема 2: 2243 док. | coh=0.592 | будут ходить, будьте внимательны, задерживаются трамваи

🔗 Залогировано в Mlflow. Run ID: 61e62246bcdf4800a50c681e607ce8ff
🏃 View run bertopic_fastapi at: http://127.0.0.1:8080/#/experiments/4/runs/61e62246bcdf4800a50c681e607ce8ff
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/4
✅ MLflow run завершен
